In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  0


In [3]:
import tensorflow as tf

# Check for GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"Num GPUs Available: {len(gpus)}")
else:
    print("No GPU available. Using CPU.")


No GPU available. Using CPU.


In [4]:
CUDA_HOME = r"C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.2"

In [5]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"Num GPUs Available: {len(gpus)}")
    for gpu in gpus:
        print(gpu)
else:
    print("No GPU available. Using CPU.")


No GPU available. Using CPU.


In [6]:
import tensorflow as tf
print(tf.__version__)


2.10.1


In [7]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"Num GPUs Available: {len(gpus)}")
    for gpu in gpus:
        print(gpu)
else:
    print("No GPU available. Using CPU.")


No GPU available. Using CPU.


In [8]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [9]:
real_videos_path = r'C:\Users\manik\Documents\devhack\deepfake\data\Celeb-real'
fake_videos_path = r'C:\Users\manik\Documents\devhack\deepfake\data\Celeb-synthesis'
youtube_videos_path = r'C:\Users\manik\Documents\devhack\deepfake\data\YouTube-real'

# Video listing
real_videos = [os.path.join(real_videos_path, video) for video in os.listdir(real_videos_path)]
fake_videos = [os.path.join(fake_videos_path, video) for video in os.listdir(fake_videos_path)]
youtube_videos = [os.path.join(youtube_videos_path, video) for video in os.listdir(youtube_videos_path)]


In [10]:
video_paths = real_videos + fake_videos + youtube_videos
labels = [0] * len(real_videos) + [1] * len(fake_videos) + [0] * len(youtube_videos)

df = pd.DataFrame({'video_path': video_paths, 'label': labels})

# Data splitting
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)


In [11]:
def extract_frames(video_path, num_frames=30):
    cap = cv2.VideoCapture(video_path)
    frames = []
    count = 0

    while cap.isOpened() and count < num_frames:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (112, 112))
        frames.append(frame)
        count += 1

    cap.release()
    return np.array(frames)

# LSTM model
def build_lstm_model(num_frames=10):
    input_layer = layers.Input(shape=(num_frames, 112, 112, 3))

    x = layers.TimeDistributed(layers.Conv2D(64, (3, 3), activation='relu'))(input_layer)
    x = layers.TimeDistributed(layers.MaxPooling2D((2, 2)))(x)
    x = layers.TimeDistributed(layers.Flatten())(x)

    x = layers.LSTM(128, return_sequences=True)(x)
    x = layers.LSTM(64, return_sequences=False)(x)

    x = layers.Dense(64, activation='relu')(x)
    output_layer = layers.Dense(1, activation='sigmoid')(x)

    model = models.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Build model
lstm_model = build_lstm_model(num_frames=10)
lstm_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10, 112, 112, 3)  0         
                             ]                                   
                                                                 
 time_distributed (TimeDistr  (None, 10, 110, 110, 64)  1792     
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 10, 55, 55, 64)   0         
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, 10, 193600)       0         
 tributed)                                                       
                                                                 
 lstm (LSTM)                 (None, 10, 128)           991892

In [12]:
def data_generator(df, batch_size, num_frames=10):
    num_samples = len(df)
    while True:
        for offset in range(0, num_samples, batch_size):
            batch_samples = df.iloc[offset:offset + batch_size]
            video_sequences = []
            labels = []

            for _, row in batch_samples.iterrows():
                video_path = row['video_path']
                label = row['label']
                frames = []

                cap = cv2.VideoCapture(video_path)
                frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

                for i in np.linspace(0, frame_count - 1, num_frames, dtype=int):
                    cap.set(cv2.CAP_PROP_POS_FRAMES, i)
                    ret, frame = cap.read()
                    if ret:
                        frame = cv2.resize(frame, (112, 112))
                        frames.append(frame)
                    else:
                        frames.append(np.zeros((112, 112, 3)))

                cap.release()
                video_sequences.append(np.array(frames))
                labels.append(label)

            X = np.array(video_sequences)
            y = np.array(labels)
            yield X, y

# Training
batch_size = 4  # Increase batch size for GPU training
num_frames = 10

train_gen = data_generator(train_df, batch_size=batch_size, num_frames=num_frames)
val_gen = data_generator(val_df, batch_size=batch_size, num_frames=num_frames)

# Train model
try:
    history = lstm_model.fit(
        train_gen,
        steps_per_epoch=len(train_df) // batch_size,
        validation_data=val_gen,
        validation_steps=len(val_df) // batch_size,
        epochs=10,
        verbose=1
    )

    # Save model
    model_save_path = 'lstm_model.h5'
    lstm_model.save(model_save_path)
    print(f"Model saved to {model_save_path}")

except Exception as e:
    print(f"Error during model training: {e}")

Epoch 1/10
  30/1175 [..............................] - ETA: 37:01 - loss: 0.4383 - accuracy: 0.8667

KeyboardInterrupt: 

In [1]:
# Create a validation generator
val_gen = data_generator(val_df, batch_size=batch_size, num_frames=10)

# Evaluate the model
val_loss, val_accuracy = lstm_model.evaluate(
    val_gen,
    steps=len(val_df) // batch_size,
    verbose=1
)

print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')

NameError: name 'data_generator' is not defined

In [14]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np

# Load the saved model
model_path = 'lstm_model.h5'
lstm_model = load_model(model_path)

In [15]:
# Create a validation generator
val_gen = data_generator(val_df, batch_size=batch_size, num_frames=10)

# Evaluate the model
val_loss, val_accuracy = lstm_model.evaluate(
    val_gen,
    steps=len(val_df) // batch_size,
    verbose=1
)

print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')

130/130 [==============================] - 56s 426ms/step - loss: 0.3841 - accuracy: 0.8712
Validation Loss: 0.3841301500797272
Validation Accuracy: 0.8711538314819336


In [16]:
from sklearn.metrics import classification_report

# Make predictions on the validation set
y_true = []
y_pred = []

# Get the true labels and predicted labels
for _ in range(len(val_df) // batch_size):
    X_batch, y_batch = next(val_gen)  # Get a batch of validation data
    preds = lstm_model.predict(X_batch)
    y_true.extend(y_batch)
    y_pred.extend((preds > 0.5).astype(int))  # Convert probabilities to binary predictions

# Generate classification report
print(classification_report(y_true, y_pred))

1/1 [==============================] - 0s 204ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        67
           1       0.87      1.00      0.93       453

    accuracy                           0.87       520
   macro avg       0.44      0.50      0.47       520
weighted avg       0.76      0.87      0.81       520



c:\Users\manik\Documents\devhack\deepfake\deepfake_env\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manik\Documents\devhack\deepfake\deepfake_env\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manik\Documents\devhack\deepfake\deepfake_env\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [17]:
lstm_model.save('deepfake_detection_model.h5')

In [28]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

def preprocess_video(video_path, frame_size=(112, 112), num_frames=10):
    # Load the video
    cap = cv2.VideoCapture(video_path)
    
    frames = []
    while len(frames) < num_frames:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, frame_size)
        frames.append(frame)

    cap.release()

    if len(frames) < num_frames:
        raise ValueError(f"Not enough frames in the video. Required: {num_frames}, Found: {len(frames)}")

    # Convert to numpy array and normalize
    frames = np.array(frames).astype('float32') / 255.0
    
    # Expand dimensions to match input shape (1, num_frames, height, width, channels)
    frames = np.expand_dims(frames, axis=0)

    return frames

# Load the previously saved model
model_path = 'C:\\Users\\manik\\Documents\\devhack\\deepfake\\deepfake_detection_model.h5'
lstm_model = load_model(model_path)

# Path to your video file
video_path = r"C:\Users\manik\Documents\devhack\deepfake\data\YouTube-real\00000.mp4"

try:
    # Preprocess the video
    input_frames = preprocess_video(video_path)

    # Make a prediction
    prediction = lstm_model.predict(input_frames)

    # Interpret the prediction
    if prediction[0][0] > 0.5:
        print("The video is classified as fake.")
    else:
        print("The video is classified as real.")

except FileNotFoundError as e:
    print(e)
except ValueError as ve:
    print(ve)

1/1 [==============================] - 1s 772ms/step
The video is classified as fake.


In [42]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

def preprocess_video(video_path, frame_size=(112, 112), num_frames=10):
    """
    Preprocess a video by extracting frames, resizing them, and normalizing the pixel values.
    
    Args:
    video_path (str): The path to the video file.
    frame_size (tuple): The desired frame size for the model input.
    num_frames (int): The number of frames to extract from the video.

    Returns:
    np.ndarray: A numpy array of shape (1, num_frames, height, width, channels)
                containing the preprocessed video frames.
    """
    # Open the video
    cap = cv2.VideoCapture(video_path)
    
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Calculate frame indices to capture evenly distributed frames
    frame_indices = np.linspace(0, total_frames - 1, num_frames).astype(int)
    
    for i in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, frame_size)
            frames.append(frame)
        else:
            frames.append(np.zeros((frame_size[0], frame_size[1], 3)))  # Add zero frames if not enough frames

    cap.release()

    if len(frames) < num_frames:
        raise ValueError(f"Not enough frames in the video. Required: {num_frames}, Found: {len(frames)}")

    # Convert to numpy array and normalize pixel values to range [0, 1]
    frames = np.array(frames).astype('float32') / 255.0
    
    # Expand dimensions to match input shape (1, num_frames, height, width, channels)
    frames = np.expand_dims(frames, axis=0)

    return frames

# Load the saved model
model_path = r'C:\Users\manik\Documents\devhack\deepfake\lstm_model.h5'
lstm_model = load_model(model_path)

# Path to the video file for prediction
video_path = r"C:\Users\manik\OneDrive\Desktop\amir khan.mp4"

try:
    # Preprocess the video
    input_frames = preprocess_video(video_path)

    # Make a prediction
    prediction = lstm_model.predict(input_frames)

    # Interpret the prediction (threshold = 0.5)
    if prediction[0][0] > 0.5:
        print(f"The video is classified as fake with confidence {prediction[0][0]:.2f}.")
    else:
        print(f"The video is classified as real with confidence {1 - prediction[0][0]:.2f}.")

except FileNotFoundError as e:
    print(f"File not found: {e}")
except ValueError as ve:
    print(f"Value Error: {ve}")


1/1 [==============================] - 1s 613ms/step
The video is classified as fake with confidence 0.87.
